In [1]:
%matplotlib widget
%config InlineBackend.figure_format = 'svg'

In [2]:
import addict
import copy
import datetime
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from importlib import reload

# Plotting the global model is much much faster with tex fonts turned off
plt.rcParams['text.usetex'] = False

In [3]:
import celeri_closure
celeri_closure = reload(celeri_closure)
import celeri
celeri = reload(celeri)

In [4]:
%%time
RUN_NAME = datetime.datetime.now().strftime("%y%m%d%H%M%S") + os.sep
command_file_name = './data/western_north_america/western_north_america_command.json'
command_file_name = './data/global/global_command.json'
command, segment, block, meshes, station, mogi, sar = celeri.read_data(command_file_name)
station = celeri.process_station(station, command)
segment = celeri.process_segment(segment, command, meshes)
sar = celeri.process_sar(sar, command)
closure = celeri.assign_block_labels(segment, station, block, mogi, sar)

CPU times: user 20.1 s, sys: 2.12 s, total: 22.2 s
Wall time: 22.2 s


In [5]:
(station['block_label'] == -1).sum()

0

In [6]:
celeri.plot_block_labels(segment, block, station, closure)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
closure.polygons[228].bounds

Polygon(edge_idxs=[3590, 15375, 15376, 15378, 15380, 15382, 15385, 15387, 15389, 15391, 15777, 15799, 15821, 15843, 15865, 15871, 15873, 15875, 15877, 15779, 15781, 15783, 15785, 15787, 15789, 15791, 15793, 15795, 15797, 15801, 15803, 15805, 15807, 7670, 7668, 7666, 7664, 8024, 8022, 8020, 8018, 8016, 8015, 8013, 8011, 8009, 8007, 8005, 8003, 8001, 7999, 15819, 15823, 15825, 15827, 15829, 15831, 15833, 15835, 15837, 15839, 15841, 15845, 15847, 15849, 15851, 15853, 15855, 15857, 15859, 15861, 15863, 15867, 15869, 16933, 16935, 16567, 16569, 16571, 16573, 16575, 16577, 16579, 16581, 16585, 16587, 16589, 16591, 16593, 16595, 16597, 16599, 16601, 16603, 16607, 16609, 16611, 16613, 16615, 16617, 16619, 16621, 16622, 16625, 16629, 16631, 16633, 16635, 16637, 16639, 16641, 16642, 16645, 16647, 16651, 16653, 16655, 16657, 16659, 16661, 12727, 12753, 12855, 13047, 13189, 13211, 13225, 13227, 13229, 12729, 12731, 12733, 12735, 12737, 12739, 12741, 12743, 12745, 12747, 12755, 12777, 12801, 12811,

In [78]:
vertices = closure.polygon_vertices[6]

In [93]:
maxs = closure.polygon_vertices_xyz[6].max(axis=0)
mins = closure.polygon_vertices_xyz[6].min(axis=0)

In [100]:
np.all((closure.polygon_vertices_xyz[6] <= maxs[None,:]) & (closure.polygon_vertices_xyz[6] >= mins[None,:]), axis=1)

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True])

In [5]:
assembly = addict.Dict()
operators = addict.Dict()
assembly = celeri.merge_geodetic_data(assembly, station, sar)
assembly, operators.block_motion_constraints = celeri.block_constraints(assembly, block, command)
assembly, operators.slip_rate_constraints = celeri.slip_rate_constraints(assembly, segment, block, command)

Isolating slip rate constraints
- Tensile-slip rate constraint on 11ba: rate = 0.00 (mm/yr), 1-sigma uncertainty = +/-1.00 (mm/yr)
- Tensile-slip rate constraint on 11bb: rate = 0.00 (mm/yr), 1-sigma uncertainty = +/-1.00 (mm/yr)
- Dip-slip rate constraint on AN_Iran_fault_2: rate = 0.00 (mm/yr), 1-sigma uncertainty = +/-3.00 (mm/yr)
- Dip-slip rate constraint on AN_Iran_fault_6: rate = 1.50 (mm/yr), 1-sigma uncertainty = +/-1.00 (mm/yr)
- Tensile-slip rate constraint on AWAT_fault_3: rate = 0.00 (mm/yr), 1-sigma uncertainty = +/-5.00 (mm/yr)
- Strike-slip rate constraint on Akar_connect2: rate = -8.00 (mm/yr), 1-sigma uncertainty = +/-5.00 (mm/yr)
- Dip-slip rate constraint on BOOB_fault_1: rate = 0.00 (mm/yr), 1-sigma uncertainty = +/-10.00 (mm/yr)
- Dip-slip rate constraint on BOOB_fault_4: rate = 0.00 (mm/yr), 1-sigma uncertainty = +/-10.00 (mm/yr)
- Tensile-slip rate constraint on BTFZ_Baldwin12b: rate = 0.00 (mm/yr), 1-sigma uncertainty = +/-10.00 (mm/yr)
- Tensile-slip rate cons

In [6]:
command = addict.Dict(command)